# HW1_ex_5_1

In [59]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def sigmoid(x):

    return  1/(1+np.exp(-x))

def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)

class NN(nn.Module):
    def __init__(self,input_size,num_classes):  
        
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,3)
        self.fc2=nn.Linear(3,num_classes)


    def forward(self,x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
    

p = np.array([[[0], [-1],[2], [-1]],
              [[0], [1], [0], [-1]],
              [[0], [2], [-1], [1]]])
label = np.array([0, 0, 1, 1])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = NN(input_size=3, num_classes=1).to(device)

mu, sigma = 0, 1 # mean and standard deviation
W1 = np.random.normal(mu, sigma, (3,3))
b1 = np.random.normal(mu, sigma, (3,1))
W2 = np.random.normal(mu, sigma, (1,3))
b2 = np.random.normal(mu, sigma, (1,1))


model.fc1.weight.data=torch.tensor(W1,dtype=float)
model.fc1.bias.data=torch.tensor(b1[:,0],dtype=float)
model.fc2.weight.data=torch.tensor(W2,dtype=float)
model.fc2.bias.data=torch.tensor(b2[:,0],dtype=float)

criterion = nn.CrossEntropyLoss()

output_batch1 = np.zeros((1,4))
for i in range(p.shape[1]):
    n1 = np.matmul(W1,p[:,i,:])+b1
    a1 = sigmoid(n1)
    n2 = np.matmul(W2,a1)+b2
    output_batch1[0,i] = n2

loss_numpy = BinaryCrossEntropy(label.reshape(-1, 1), output_batch1[0,:].reshape(-1,1))

output_batch2 = []
loss_torch = []
for i in range(p.shape[1]):
    out = model(torch.tensor(p[:,i,:],dtype=float).reshape((1,3)))
    l=criterion(out,torch.tensor([[label[i]]],dtype=float))
    loss_torch.append(l.item())
    output_batch2.append(out.item())
print(f"Linear for first layer")
print(output_batch1)
print(output_batch2)
print(loss_torch,"loss in pytorch")


Linear for first layer
[[-0.77326505 -1.25342003 -0.40436417 -1.34340503]]
[-0.7732650458454972, -1.2534200288085957, -0.404364166820751, -1.343405032225237]
[-0.0, -0.0, -0.0, -0.0] loss in pytorch


C:\Users\TUF\AppData\Local\Temp\ipykernel_16816\4261261754.py:60: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output_batch1[0,i] = n2


# HW1_ex_5_2

In [82]:
import numpy as np

def Hardlim(inputs):
        output = np.zeros((inputs.shape))
        output[np.where(inputs>0)] = 1
        return output

def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)

def sigmoid(x):

    return  1/(1+np.exp(-x))

# input
p = np.array([[[-1], [-1], [0], [1],[2],[2], [1], [0]],
              [[1], [0], [2], [2],[0],[1], [-1], [-1]]])

label = np.array([[[1], [1], [0], [0], [0], [0], [0], [0]],
                  [[0], [0], [1], [1], [0], [0], [0], [0]],
                  [[0], [0], [0], [0], [1], [1], [0], [0]],
                  [[0], [0], [0], [0], [0], [0], [1], [1]],])

# defining weights and biases for first layer

W = np.ones((4,2))
b = np.ones((4,1))

## accuracy function 
def acc_calc(p,W,b,label):
    
    correct_t=0
    for j in range(p.shape[1]):
        correct=0
        n = np.matmul(W,p[:,j,:])+b
        a = sigmoid(n)

        diff=np.abs(label[:,j,:]-a)
        
        
        for i in diff:
            
            if i<=0.1:
                
                correct += 1
        
        if correct==label.shape[0]:
            correct_t+=1
        

    # accuracy
    accuracy = (correct_t/8)*100 
    return accuracy   

# Train
epoch = 0
alpha = 0.9
accuracy = 0
while accuracy<100.0:
  
    for j in range(p.shape[1]):

        n = np.matmul(W,p[:,j,:])+b
        a = sigmoid(n)
        e = label[:,j,:]-a
        s = e # for binary cross entropy in output layer
        dw = alpha*s*p[:,j,:].T
        db = alpha*s
        W = W + dw
        b = b + db
    
    accuracy = acc_calc(p,W,b,label)
    
    epoch += 1
    print(f"accuracy for epoch{epoch} is {accuracy}")


# test
p1 = np.array([[[0], [0], [1], [-2]],
              [[0], [1], [-2], [5]]])
for j in range(p1.shape[1]):

        n = np.matmul(W,p1[:,j,:])+b
        a = sigmoid(n)
        print(f"output for {p1[:,j,:]} is:\n") 
        print(f"{a}\n")

accuracy for epoch1 is 0.0
accuracy for epoch2 is 0.0
accuracy for epoch3 is 0.0
accuracy for epoch4 is 0.0
accuracy for epoch5 is 0.0
accuracy for epoch6 is 0.0
accuracy for epoch7 is 0.0
accuracy for epoch8 is 0.0
accuracy for epoch9 is 0.0
accuracy for epoch10 is 0.0
accuracy for epoch11 is 12.5
accuracy for epoch12 is 25.0
accuracy for epoch13 is 25.0
accuracy for epoch14 is 50.0
accuracy for epoch15 is 50.0
accuracy for epoch16 is 62.5
accuracy for epoch17 is 62.5
accuracy for epoch18 is 62.5
accuracy for epoch19 is 62.5
accuracy for epoch20 is 62.5
accuracy for epoch21 is 62.5
accuracy for epoch22 is 62.5
accuracy for epoch23 is 62.5
accuracy for epoch24 is 62.5
accuracy for epoch25 is 62.5
accuracy for epoch26 is 62.5
accuracy for epoch27 is 62.5
accuracy for epoch28 is 62.5
accuracy for epoch29 is 75.0
accuracy for epoch30 is 87.5
accuracy for epoch31 is 87.5
accuracy for epoch32 is 87.5
accuracy for epoch33 is 87.5
accuracy for epoch34 is 87.5
accuracy for epoch35 is 87.5
accu

In [88]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def sigmoid(x):

    return  1/(1+np.exp(-x))

def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)

class NN(nn.Module):
    def __init__(self,input_size):  
        
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,4)


    def forward(self,x):
        x = F.sigmoid(self.fc1(x))
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = NN(input_size=2).to(device)

model.fc1.weight.data=torch.tensor(W,dtype=float)
model.fc1.bias.data=torch.tensor(b[:,0],dtype=float)

model.fc1.weight
output_batch2 = []

for i in range(p1.shape[1]):
    out = model(torch.tensor(p1[:,i,:],dtype=float).reshape((1,2)))
    
    print(out)


tensor([[0.0346, 0.0004, 0.0003, 0.0362]], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)
tensor([[3.2029e-02, 8.0341e-02, 3.2743e-04, 3.0631e-05]], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)
tensor([[3.6758e-05, 4.4838e-09, 7.8319e-02, 9.9998e-01]], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)
tensor([[9.9997e-01, 1.0000e+00, 5.0048e-09, 1.6035e-17]], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)


# HW1_ex_5_3

In [96]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def mse_loss(y_true,y_predict):
    return np.mean((y_true-y_predict)**2,axis = -1)

# input
p = np.array([[[0], [-4],[1.7], [-2]],
              [[1], [3], [-1], [-3]],
              [[2], [-3], [0], [1.5]]])

label = np.array([[[0.4], [-1.6], [3.7], [-0.9]]])
print(label.shape)


# defining weights and biases for first layer

W1 = np.ones((4,3))
b1 = np.ones((4,1))
W2 = np.ones((1,4))*-1.0
b2 = np.ones((1,1))

# forward prop
output_batch = np.zeros((1,4,1))
for i in range(p.shape[1]):
    n1 = np.matmul(W1,p[:,i,:])+b1
    a1 = np.maximum(0,n1)
    n2 = np.matmul(W2,a1)+b2
    output_batch[:,i,:] = n2

# calculating loss and accuracy for each input

loss=[]
for i in range(output_batch.shape[1]):

    loss.append(mse_loss(label[:,i,:],output_batch[:,i,:])[0])


print(f"loss={loss}\n")
print(f"mean loss = {(loss[0]+loss[1]+loss[2]+loss[3])/4}")

(1, 4, 1)
loss=[237.16000000000003, 6.760000000000001, 90.25, 3.61]

mean loss = 84.44500000000001


In [97]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def sigmoid(x):

    return  1/(1+np.exp(-x))

def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)

class NN(nn.Module):
    def __init__(self,input_size):  
        
        super(NN,self).__init__()
        self.fc1=nn.Linear(input_size,3)
        self.fc2=nn.Linear(3,1)


    def forward(self,x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = NN(input_size=3).to(device)

model.fc1.weight.data=torch.tensor(W1,dtype=float)
model.fc1.bias.data=torch.tensor(b1[:,0],dtype=float)
model.fc2.weight.data=torch.tensor(W2,dtype=float)
model.fc2.bias.data=torch.tensor(b2[:,0],dtype=float)

criterion = nn.MSELoss()

model.fc1.weight
output_batch2 = []

for i in range(p1.shape[1]):
    out = model(torch.tensor(p[:,i,:],dtype=float).reshape((1,3)))
    l=criterion(out,torch.tensor([[label[:,i,:]]],dtype=float))
    print(l)

tensor(11.0760, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(5.8095, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(36.9924, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(2.5490, dtype=torch.float64, grad_fn=<MseLossBackward0>)
